In [43]:
import requests
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import klib
from pint import UnitRegistry

ureg = UnitRegistry()
ureg.default_format = '.2f'
ureg.load_definitions('crypto_mining_units.txt')

pd.set_option("display.max.columns", None)


In [37]:
def get_network_data(asset: str, metrics: list = ['HashRate', 'IssTotUSD', 'FeeTotUSD', 'PriceUSD', 'FeeMeanNtv'], end_time: str = date.today().strftime('%Y-%m-%d')):
    api_str = f"https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?assets={asset}&metrics={','.join(metrics)}&end_time={end_time}&page_size=10000&pretty=true"
    response = requests.get(api_str).json()
    return pd.DataFrame(response['data'])

In [38]:
df_unclean = get_network_data("btc", ['HashRate', 'PriceUSD', 'FeeMeanNtv']).fillna("0")

df_cleaned = klib.data_cleaning(df_unclean)

df_cleaned.time = df_cleaned.time.str.replace("T00:00:00.000000000Z", "")
df_cleaned.time = pd.to_datetime(df_cleaned.time)

df_cleaned[['hash_rate', 'price_usd']] =  df_cleaned[['hash_rate', 'price_usd']].apply(pd.to_numeric)

df_cleaned.index = df_cleaned.time
df_cleaned.drop(columns=['time'], inplace=True)

df_cleaned.dtypes
df = df_cleaned
df.to_csv("hell")
df

Shape of cleaned data: (4567, 4)Remaining NAs: 0

Changes:
Dropped rows: 0
     of which 0 duplicates. (Rows: [])
Dropped columns: 1
     of which 1 single valued.     Columns: ['asset']
Dropped missing values: 0
Reduced memory by at least: -0.16 MB (--94.12%)



,fee_mean_ntv,hash_rate,price_usd
time,,,
2009-01-09,0,9.444951e-07,0.000000
2009-01-10,0,3.032326e-06,0.000000
2009-01-11,0,4.623055e-06,0.000000
2009-01-12,0,4.672765e-06,0.000000
2009-01-13,0,6.114363e-06,0.000000
...,...,...,...
2021-07-07,0.00017001,8.782076e+07,33931.618638
2021-07-08,0.00012762,9.924459e+07,32819.659531
2021-07-09,0.00011726,9.353267e+07,33914.881074


In [39]:
def get_latest_price():
    return float(df.price_usd.tail(1).values)
def get_latest_hashrate():
    return float(df.hash_rate.tail(1).values)
def get_latest_tx_fees():
    return float(df.fee_mean_ntv.tail(1).values[0])

https://www.aniccaresearch.tech/blog/the-alchemy-of-hashpower-part-i

D = C / [P * S/H * (m+n) * 6 * 24 - k * S * r * 24]

In [44]:
# Calculating days to break even for one S19j Pro 
capex       = 5_500 * ureg.USD
opex        = 0.1   * ureg.USD / ureg.kWh
asset_price = get_latest_price() * ureg.USD / ureg.BTC

miner_hashrate   = 96 * ureg.TH / ureg.second  
network_hashrate = get_latest_hashrate() * ureg.TH / ureg.s
miner_efficency  = 29.5 * ureg.J / ureg.TH 
hashrate_share = miner_hashrate.to('hashes / second') / network_hashrate.to('hashes / second')

subsidy      = 6.25 * ureg.BTC / ureg.BTC_block
tx_fees      = get_latest_tx_fees() * ureg.BTC / ureg.BTC_block
block_reward = subsidy + tx_fees
blocks       = 144 * ureg.BTC_block / ureg.day

profits = asset_price * hashrate_share * block_reward * blocks
losses  = miner_efficency.to('kWh / terahashes') * miner_hashrate.to('terahashes / day') * opex 
break_even = capex / (profits - losses)

print(f'{break_even} to break even with one S19j Pro with Income {profits} and Electricity Costs {losses}')


208.58 day to break even with one S19j Pro with Income 33.17 USD / day and Electricity Costs 6.80 USD / day
